### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Anxiety_disorder/GSE60190'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Step 1: Gene Expression Data Availability
is_gene_available = True  # Since Illumina HumanHT-12 v3 microarray is used for gene expression

# Step 2: Variable Availability and Data Type Conversion

# For Anxiety_disorder
if len(set([item.split(': ')[-1] for item in sample_characteristics[3]])) > 1:
    trait_row = 3  # Anxiety disorder likely mapped to diagnostic codes (dx)
else:
    trait_row = None

# For age
if len(set([item.split(': ')[-1] for item in sample_characteristics[5]])) > 1:
    age_row = 5
else:
    age_row = None

# For gender
if len(set([item.split(': ')[-1] for item in sample_characteristics[7]])) > 1:
    gender_row = 7
else:
    gender_row = None

# Conversion functions
def convert_trait(value):
    mapping = {
        'OCD': 1,
        'Control': 0,  # assuming Control as 0 for binary conversion
        'ED': None,  # We are focusing on Anxiety_disorder only, thus mapping is not required
        'Bipolar': None,
        'MDD': None,
        'Tics': None,
    }
    key_value = value.split(': ')[-1]
    return mapping.get(key_value, None)

def convert_age(value):
    try:
        return float(value.split(': ')[-1])
    except ValueError:
        return None
        
def convert_gender(value):
    key_value = value.split(': ')[-1]
    return 0 if key_value == 'F' else 1 if key_value == 'M' else None

# Save Metadata
save_cohort_info('GSE60190', './preprocessed/Anxiety_disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Anxiety_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Anxiety_disorder/trait_data/GSE60190.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
